In [1]:
from theano import tensor

from blocks.algorithms import GradientDescent, Scale
from blocks.bricks import MLP, Tanh, Softmax
from blocks.bricks.cost import CategoricalCrossEntropy, MisclassificationRate
from blocks.initialization import IsotropicGaussian, Constant
from fuel.streams import DataStream
from fuel.transformers import Flatten
from fuel.datasets import MNIST
from fuel.schemes import SequentialScheme
from blocks.filter import VariableFilter
from blocks.graph import ComputationGraph
from blocks.model import Model
from blocks.monitoring import aggregation
from blocks.extensions import FinishAfter, Timing, Printing
from blocks.extensions.saveload import Checkpoint
from blocks.extensions.monitoring import (DataStreamMonitoring,
                                          TrainingDataMonitoring)
from blocks.main_loop import MainLoop
from blocks.roles import WEIGHT


In [2]:
mlp = MLP([Tanh(), Softmax()], [784, 100, 10],
          weights_init=IsotropicGaussian(0.01),
          biases_init=Constant(0))
mlp.initialize()
x = tensor.matrix('features')
y = tensor.lmatrix('targets')
probs = mlp.apply(x)
cost = CategoricalCrossEntropy().apply(y.flatten(), probs)
error_rate = MisclassificationRate().apply(y.flatten(), probs)



In [3]:
train_set = MNIST(('train',))

In [4]:
data_stream = Flatten(DataStream.default_stream(train_set,iteration_scheme=SequentialScheme(train_set.num_examples,500)),
                      which_sources=('features'))

In [5]:
train_x, train_y = next(data_stream.get_epoch_iterator())

In [6]:
train_x.shape

(500, 784)

In [8]:
p = probs.eval({x:train_x})

In [9]:
p.shape

(500, 10)

In [13]:
e = error_rate.eval({y:train_y,probs:p})

In [14]:
e

array(0.9340000152587891, dtype=float32)